In [13]:
import fitz
import pandas as pd
import cv2
import json
# allow pandas to display everything in ipynb
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

doc_id = "f654f716-0693-4888-9d87-092f2476195b"

metadata_path = f"/Users/bigo/Projects/timbergem/data/processed/{doc_id}/symbols/symbols_metadata.json"
document_path = f"/Users/bigo/Projects/timbergem/data/processed/{doc_id}/original.pdf"
legend_path = f"/Users/bigo/Projects/timbergem/data/processed/{doc_id}/clippings/page1/SymbolLegend_1_clipping.png"

"""
Debug the symbol coordinates in the legend by loading the legend image and applying the coordinates in metadata.symbols[].coordinates.legend_relative (utilizing left, top, width, height). Saves a clipping from the legend image in "symbol_debug"
"""
def debug_symbol_coordinates():
    with open(metadata_path, "r") as f:
        metadata_json = json.load(f)
    symbols = metadata_json["symbols"]
    metadata = pd.DataFrame(symbols)
    legend_image = cv2.imread(legend_path)

    for i, symbol in metadata.iterrows():
        coordinates = symbol['coordinates']['legend_relative']
        left = coordinates['left']
        top = coordinates['top']
        width = coordinates['width']
        height = coordinates['height']
        # extract the symbol clipping from the legend image
        symbol_clipping = legend_image[top:top + height, left:left + width]
        # save the symbol clipping as an image
        symbol_image_path = f"symbol_debug/symbol_{i}.png"
        cv2.imwrite(symbol_image_path, symbol_clipping)
        print(f"Saved symbol clipping to {symbol_image_path}")
debug_symbol_coordinates()


Saved symbol clipping to symbol_debug/symbol_0.png


Testing coordinate transformation with fixed function:
Canvas coords: {'left': 355.75, 'top': 90.43, 'width': 103, 'height': 103}
Canvas dimensions: 1200x800
       → DETAILED Coordinate transformation debug:
         INPUT - Canvas coords: (355.8, 90.4) 103.0x103.0
         INPUT - Canvas dimensions: 1200.0x800.0
         METADATA - PDF: 2592.0x1728.0 pts
         METADATA - Image: 7200x4800 px
         METADATA - Rotation: 0°
         METADATA - DPI: 200
         SCALING - Expected frontend canvas: 1200.0x800.0
         SCALING - Calculated scale: 0.166667
         SCALING - Image aspect ratio: 1.500000
         ORIGINAL IMAGE - Coords: (2134.5, 542.6) 618.0x618.0
         FINAL PDF - Coords: (768.4, 1310.2) 222.5x222.5
PDF coords: {'left': 768.42, 'top': 1310.1912, 'width': 222.48, 'height': 222.48}
Expected PDF coords: {'left': 128.07, 'top': 32.55, 'width': 37.08, 'height': 37.08}
Difference in left: 640.35
Difference in top: 1277.64
Difference in width: 185.40
Difference in heigh

In [14]:

"""
calculates the symbol dimensions in 300 dpi based on the contours of the symbol clippings
references each symbol in the metadata json "symbols" list. Utilizes "source_legend.page_number"(1-indexed) and "coordinates.pdf_absolute" to find the symbol clipping in the pdf.
prints the height and width of each symbol
saves the contour clippings in "symbol_debug" dir in root of the project
"""
def calculate_symbol_dimensions():
    with open(metadata_path, "r") as f:
        metadata_json = json.load(f)
    symbols = metadata_json["symbols"]
    metadata = pd.DataFrame(symbols)
    document = fitz.open(document_path)

    for i, symbol in metadata.iterrows():
        page_number = symbol['source_legend']['page_number'] - 1  # convert to 0-indexed
        coordinates = symbol['coordinates']['pdf_absolute']
        # pdf_absolute contains left, top, width, and height
        left = coordinates['left']
        top = coordinates['top']
        width = coordinates['width']
        height = coordinates['height']
        # calculate the dimensions in 300 dpi
        dpi = 300
        width_in_inches = width / dpi
        height_in_inches = height / dpi
        print(f"Symbol {i}: Width: {width_in_inches:.2f} inches, Height: {height_in_inches:.2f} inches")
        # calculate the dimensions in 300dpi pixels
        width_in_pixels = int(width * dpi / 72)
        height_in_pixels = int(height * dpi / 72)
        print(f"Symbol {i}: Width: {width_in_pixels} pixels, Height: {height_in_pixels} pixels")
        # extract the symbol clipping from the pdf
        page = document[page_number]
        rect = fitz.Rect(left, top, left + width, top + height)
        pix = page.get_pixmap(clip=rect, dpi=dpi)
        # save the symbol clipping as an image
        symbol_image_path = f"symbol_debug/symbol_{i}.png"
        pix.save(symbol_image_path)
        print(f"Saved symbol clipping to {symbol_image_path}")

calculate_symbol_dimensions()


Symbol 0: Width: 0.32 inches, Height: 0.12 inches
Symbol 0: Width: 398 pixels, Height: 146 pixels
Saved symbol clipping to symbol_debug/symbol_0.png
